# Regresión para Iris DataSet

In [7]:
using Flux

using Flux: crossentropy, normalise, onecold, onehotbatch
using Statistics: mean

In [12]:
labels = Flux.Data.Iris.labels()
features = Flux.Data.Iris.features()

4×150 Array{Float64,2}:
 5.1  4.9  4.7  4.6  5.0  5.4  4.6  5.0  …  6.8  6.7  6.7  6.3  6.5  6.2  5.9
 3.5  3.0  3.2  3.1  3.6  3.9  3.4  3.4     3.2  3.3  3.0  2.5  3.0  3.4  3.0
 1.4  1.4  1.3  1.5  1.4  1.7  1.4  1.5     5.9  5.7  5.2  5.0  5.2  5.4  5.1
 0.2  0.2  0.2  0.2  0.2  0.4  0.3  0.2     2.3  2.5  2.3  1.9  2.0  2.3  1.8

#### Normalizamos los datos
* Restamos el promedio, dividimos por el desvío estándard 
* queremos un promedio normalziado de 0 y un desvío de 1.


In [13]:
normed_features = normalise(features, dims=2)

4×150 Array{Float64,2}:
 -0.900681  -1.14302   -1.38535   …   0.795669  0.432165   0.0686618
  1.03206   -0.124958   0.337848     -0.124958  0.800654  -0.124958 
 -1.34127   -1.34127   -1.39814       0.819624  0.933356   0.762759 
 -1.31298   -1.31298   -1.31298       1.05354   1.44796    0.790591 

One Hot Enoding de las labels

In [14]:
klasses = sort(unique(labels))
onehot_labels = onehotbatch(labels, klasses)

3×150 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
  true   true   true   true   true  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true   true   true   true   true

Separamos en train y test sets

In [15]:
train_indices = [1:3:150 ; 2:3:150]

X_train = normed_features[:, train_indices]
y_train = onehot_labels[:, train_indices]

X_test = normed_features[:, 3:3:150]
y_test = onehot_labels[:, 3:3:150]

3×50 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
  true   true   true   true   true  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true   true   true   true   true

## Modelo
Armamos un modelo usando Chain, toma 4 features como input y devuelve 3 probabilidades

In [8]:
model = Chain(
    Dense(4, 3),
    softmax
)

Chain(Dense(4, 3), NNlib.softmax)

Definimos nuestra función de loss

In [9]:
loss(x, y) = crossentropy(model(x), y)

loss (generic function with 1 method)

Optimizador para gradient descent con learning rate de 0.5

In [10]:
optimiser = Descent(0.5)


Descent(0.5)

Iterador para entrenar el modelo en 110 épocas

In [16]:
data_iterator = Iterators.repeated((X_train, y_train), 110)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}(([-0.900681 -1.50652 … 1.038 0.432165; 1.03206 0.106445 … -0.124958 0.800654; -1.34127 -1.28441 … 0.819624 0.933356; -1.31298 -1.31298 … 1.44796 1.44796], Bool[true true … false false; false false … false false; false false … true true])), 110)

In [18]:
println("Entrenado.")
Flux.train!(loss, params(model), data_iterator, optimiser)

Entrenado.


### Model Evaluation

In [19]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
accuracy_score = accuracy(X_test, y_test)
println("\nAccuracy: $accuracy_score")



Accuracy: 0.92


### Matriz de confusión

In [20]:
function confusion_matrix(X, y)
    ŷ = onehotbatch(onecold(model(X)), 1:3)
    y * ŷ'
end

confusion_matrix (generic function with 1 method)

In [22]:

display(confusion_matrix(X_test, y_test))

3×3 Array{Int64,2}:
 15   1   0
  0  16   1
  0   2  15